<a href="https://colab.research.google.com/github/Mugeshkumar133/Singapore-Flat-Resale-Prediction/blob/main/Singapore_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import pandas as pd
import json
import requests

In [ ]:
df1=pd.read_csv("/content/ResaleFlatPricesBasedonApprovalDate19901999.csv")
df2=pd.read_csv("/content/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
df3=pd.read_csv("/content/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
df4=pd.read_csv("/content/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")
df5=pd.read_csv("/content/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
df = pd.concat([df1 , df2 , df3, df4, df5],axis=0)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977.0,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977.0,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977.0,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977.0,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976.0,47200.0,NaN


In [ ]:
df.shape

(188584, 11)

Handling Null Values

In [ ]:
df.isnull().sum()

month                       0
town                        0
flat_type                   1
block                       1
street_name                 1
storey_range                2
floor_area_sqm              3
flat_model                  3
lease_commence_date         3
resale_price                3
remaining_lease        119224
dtype: int64

In [ ]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

#MRT (Mass Rapid Transit System) Railway Transportation


*   Getting MRT coordinates so that we can calculate the distance afterwards using MRT coordinates



In [ ]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [ ]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [ ]:
mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal=200640&returnGeom=Y&getAddrDetails=Y&pageNum=1'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)

    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Bukit Batok MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Bukit Gombak MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Choa Chu Kang MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Yew Tee MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Kranji MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Marsiling MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Woodlands MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Admiralty MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Sembawang MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Canberra MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Yishun MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Khatib MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Yio Chu Kang MRT Station,Lat: 1.30743547948389 Long: 103.854713903431
Ang Mo Kio MRT Station,Lat: 1.30743547948389 Long: 103.85471

#Getting coordinates of each HDB(Housing and Developing Board) Resale flat

In order to conduct the distance from MRT stations as well as


*   So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System).
*   And, also from CBD (Central Business District).



In [ ]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

In [ ]:
mrt_location.to_csv("data_mart.csv")

In [ ]:
df['address'] = df['block'] + " " + df['street_name']
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986.0,255000.0,70,174 ANG MO KIO AVE 4
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981.0,275000.0,65,541 ANG MO KIO AVE 10
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980.0,285000.0,64,163 ANG MO KIO AVE 4
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979.0,290000.0,63,446 ANG MO KIO AVE 10
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980.0,290000.0,64,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
32202,2018-07,TAMPINES,4 ROOM,916,TAMPINES ST 91,01 TO 03,104.0,Model A,1984.0,413000.0,65 years 02 months,916 TAMPINES ST 91
32203,2018-07,TAMPINES,4 ROOM,138,SIMEI ST 1,04 TO 06,84.0,Simplified,1988.0,415000.0,69 years 01 month,138 SIMEI ST 1
32204,2018-07,TAMPINES,4 ROOM,154,SIMEI ST 1,07 TO 09,84.0,Simplified,1988.0,415000.0,68 years 09 months,154 SIMEI ST 1
32205,2018-07,TAMPINES,4 ROOM,464,TAMPINES ST 44,01 TO 03,104.0,Model A,1985.0,415000.0,66 years 04 months,464 TAMPINES ST 44


In [ ]:
address_list = df['address'].unique()

In [ ]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    query_string='https://www.onemap.gov.sg/api/common/elastic/search?searchVal=200640&returnGeom=Y&getAddrDetails=Y&pageNum=1'
    resp = requests.get(query_string)


    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

174 ANG MO KIO AVE 4 ,Lat: 1.30743547948389 Long: 103.854713903431
541 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.854713903431
163 ANG MO KIO AVE 4 ,Lat: 1.30743547948389 Long: 103.854713903431
446 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.854713903431
557 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.854713903431
603 ANG MO KIO AVE 5 ,Lat: 1.30743547948389 Long: 103.854713903431
709 ANG MO KIO AVE 8 ,Lat: 1.30743547948389 Long: 103.854713903431
333 ANG MO KIO AVE 1 ,Lat: 1.30743547948389 Long: 103.854713903431
109 ANG MO KIO AVE 4 ,Lat: 1.30743547948389 Long: 103.854713903431
564 ANG MO KIO AVE 3 ,Lat: 1.30743547948389 Long: 103.854713903431
218 ANG MO KIO AVE 1 ,Lat: 1.30743547948389 Long: 103.854713903431
556 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.854713903431
156 ANG MO KIO AVE 4 ,Lat: 1.30743547948389 Long: 103.854713903431
471 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.854713903431
434 ANG MO KIO AVE 10 ,Lat: 1.30743547948389 Long: 103.85

JSONDecodeError: ignored

In [ ]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

3987

In [ ]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [ ]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))


Getting the diatnce with the help of OpenMap API

In [ ]:
# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [ ]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [ ]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [ ]:
df_coordinates

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt
0,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0
1,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,541 ANG MO KIO AVE 10,2728.531219,0.0
2,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,163 ANG MO KIO AVE 4,2728.531219,0.0
3,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,446 ANG MO KIO AVE 10,2728.531219,0.0
4,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,557 ANG MO KIO AVE 10,2728.531219,0.0
...,...,...,...,...,...,...,...,...
3982,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,62 MARINE DR,2728.531219,0.0
3983,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,127 PASIR RIS ST 11,2728.531219,0.0
3984,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,102 PASIR RIS ST 12,2728.531219,0.0
3985,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,633 PASIR RIS DR 3,2728.531219,0.0


In [ ]:
df_coordinates.to_csv('df_coordinates.csv',index=False)

In [ ]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986.0,255000.0,70
1,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986.0,275000.0,69
2,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986.0,310000.0,68
3,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986.0,253000.0,68
4,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986.0,290000.0,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69355,NaN,NaN,NaN,NaN,NaN,212A PUNGGOL WALK,NaN,NaN,2018-07,PUNGGOL,4 ROOM,212A,PUNGGOL WALK,13 TO 15,93.0,Model A,2015.0,468000.0,95 years 10 months
69356,NaN,NaN,NaN,NaN,NaN,209A PUNGGOL PL,NaN,NaN,2018-07,PUNGGOL,4 ROOM,209A,PUNGGOL PL,07 TO 09,92.0,Model A,2014.0,465000.0,95 years 05 months
69357,NaN,NaN,NaN,NaN,NaN,213A PUNGGOL WALK,NaN,NaN,2018-07,PUNGGOL,5 ROOM,213A,PUNGGOL WALK,10 TO 12,113.0,Improved,2015.0,590000.0,95 years 10 months
69358,NaN,NaN,NaN,NaN,NaN,430A FERNVALE LINK,NaN,NaN,2018-07,SENGKANG,4 ROOM,430A,FERNVALE LINK,01 TO 03,92.0,Model A,2015.0,388000.0,95 years 10 months


In [ ]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,lease_remain_years
0,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,62
1,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,275000.0,69,62
2,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,310000.0,68,62
3,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,253000.0,68,62
4,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,174 ANG MO KIO AVE 4,2728.531219,0.0,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,290000.0,68,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39339,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,440 PASIR RIS DR 4,2728.531219,0.0,2015-05,PASIR RIS,4 ROOM,440,PASIR RIS DR 4,04 TO 06,103.0,Model A,1989,390000.0,73,65
39340,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,440 PASIR RIS DR 4,2728.531219,0.0,2016-06,PASIR RIS,4 ROOM,440,PASIR RIS DR 4,01 TO 03,118.0,Model A,1989,455000.0,72,65
39341,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,440 PASIR RIS DR 4,2728.531219,0.0,2016-11,PASIR RIS,4 ROOM,440,PASIR RIS DR 4,07 TO 09,118.0,Model A,1989,480000.0,71,65
39342,1.30743547948389,103.854713903431,640,ROWELL ROAD,200640,440 PASIR RIS DR 4,2728.531219,0.0,2017-05,PASIR RIS,4 ROOM,440,PASIR RIS DR 4,07 TO 09,106.0,Model A,1989,462500.0,71 years,65


In [ ]:
df_new.to_csv('combined.csv', index = False)